In [1]:
import torch
import pandas as pd
from preprocessing.preprocessing import json_to_tsv
from preprocessing.tokenizer import Mecab_Tokenizer
from data_loader import CustomDataset, Vocab
from model.transformer import Transformer

from torch.utils.data import DataLoader

In [5]:
df = json_to_tsv("/Users/simso/Project/summarization/data", "train.json")
df.head()

,uid,title,region,context,summary,total
1000,1000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의석을 정돈하여 주시기 ...
1001,1001,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의사팀장 수고하셨습니다....
1002,1002,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제2항 ...
1003,1003,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제3항 ...
1004,1004,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록 완주 의석을 정돈...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2994 entries, 1000 to 3993
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   uid      2994 non-null   object
 1   title    2994 non-null   object
 2   region   2994 non-null   object
 3   context  2994 non-null   object
 4   summary  2994 non-null   object
 5   total    2994 non-null   object
dtypes: object(6)
memory usage: 228.3+ KB


In [13]:
train = pd.read_csv("./data/train.tsv", sep='\t')

In [14]:
train.head()

,Unnamed: 0,uid,title,region,context,summary,total
0,1000,1000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의석을 정돈하여 주시기 ...
1,1001,1001,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의사팀장 수고하셨습니다....
2,1002,1002,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제2항 ...
3,1003,1003,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제3항 ...
4,1004,1004,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록 완주 의석을 정돈...


In [8]:
train_vocab = Vocab(train['total'], train['summary'], 
                    encoder_len=500, decoder_len=50, 
                    max_vocab_size=20000)

train_src_tokens = train_vocab.src_tokenizer.txt2token(train['total'])

train_tgt_tokens = train_vocab.tgt_tokenizer.txt2token(train['summary'])

from torch.utils.data import DataLoader

train_loader = DataLoader(CustomDataset(train_src_tokens, train_tgt_tokens), batch_size=64, num_workers=1, shuffle=True)


100%|██████████| 2994/2994 [00:00<00:00, 327929.86it/s]


In [19]:
mini_batch = next(iter(train_loader))
mini_batch['src'].size(), mini_batch['tgt'].size()

(torch.Size([64, 500]), torch.Size([64, 50]))

In [2]:
saved_data = torch.load('/Users/simso/Project/summarization/trained_models/transformer.bs-32.n_epoch-20.enc-500.dec-50.max_vocab-20000.adam.20.0.13-1.13.0.10-1.11.pth', map_location='cpu')

In [3]:
train_config = saved_data['config']
train_config

Namespace(batch_size=32, decoder_len=50, dropout=0.2, encoder_len=500, gpu_id=0, hidden_size=768, init_epoch=1, iteration_per_update=1, lr=0.0001, lr_decay_start=10, lr_gamma=0.5, lr_step=0, max_grad_norm=100000000.0, max_vocab_size=20000, model_fn='/content/drive/MyDrive/Projects/lg-text-summarization/trained_models/transformer.bs-32.n_epoch-20.enc-500.dec-50.max_vocab-20000.adam.pth', n_epochs=20, n_layers=6, n_splits=8, off_autocast=False, train_fn='/content/drive/MyDrive/Projects/lg-text-summarization/data/train.tsv', use_adam=True, use_radam=False, use_transformer=False, valid_size=200, verbose=2)

In [4]:
src_vocab = saved_data['src_vocab']
tgt_vocab = saved_data['tgt_vocab']

len(src_vocab), len(tgt_vocab)

(20002, 20002)

In [5]:
keys = [i for i in tgt_vocab.keys()]
keys[0]

'pad_'

In [7]:

# Initialize dataloader, but we don't need to read traniing & test corpus.
# What we need is just load vocabularies from the previously trained model.
vocab = Vocab()
vocab.set_vocab(src_vocab, tgt_vocab)

# Load text data to summarize.
data = pd.read_csv('./data/test.tsv', sep="\t", )
tokens = vocab.src_tokenizer.txt2token(data['total'])

loader = DataLoader(CustomDataset(tokens, mode='text'), batch_size=4, num_workers=1, shuffle=False)

# Get model as trained.
input_size, output_size = len(src_vocab), len(tgt_vocab)

def get_model(intput_size, output_size, train_config):
    model = Transformer(
        input_size,
        train_config.hidden_size,
        output_size,
        n_splits=train_config.n_splits,
        n_enc_blocks=train_config.n_layers,
        n_dec_blocks=train_config.n_layers,
        dropout_p=train_config.dropout,
    )
    model.load_state_dict(saved_data['model'])
    model.eval()

    return model

model = get_model(input_size, 11946, train_config)

100%|██████████| 506/506 [00:00<00:00, 19389.16it/s]


In [11]:
x = next(iter(loader))['src']


In [12]:
x.size()

torch.Size([4, 500])

In [13]:
y_hats, indices = model.search(x)

/Users/simso/Project/summarization/model/transformer.py:322: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask = torch.tensor(torch.eq(x, 0), dtype=torch.bool)


In [15]:
y_hats

tensor([[[-8.9282e+00, -2.1248e+01, -1.9175e+01,  ..., -1.7546e+01,
          -1.8773e+01, -1.9387e+01],
         [-1.3721e+01, -1.6054e+01, -1.5161e+01,  ..., -1.5735e+01,
          -1.3687e+01, -1.5669e+01],
         [-1.8474e+01, -2.4923e+01, -2.0480e+01,  ..., -2.1228e+01,
          -2.2342e+01, -2.2833e+01],
         ...,
         [-3.3379e-06, -2.8823e+01, -2.1392e+01,  ..., -2.3822e+01,
          -2.4166e+01, -2.4131e+01],
         [-2.9802e-06, -2.9017e+01, -2.1629e+01,  ..., -2.4005e+01,
          -2.4253e+01, -2.4203e+01],
         [-2.9802e-06, -2.8970e+01, -2.1689e+01,  ..., -2.3942e+01,
          -2.4091e+01, -2.4127e+01]],

        [[-4.8414e+00, -2.0594e+01, -1.8138e+01,  ..., -1.5575e+01,
          -1.7533e+01, -1.7772e+01],
         [-1.1121e+01, -1.4764e+01, -1.3461e+01,  ..., -1.2985e+01,
          -1.2160e+01, -1.3289e+01],
         [-1.1009e+01, -1.3060e+01, -1.3180e+01,  ..., -1.1046e+01,
          -1.3680e+01, -1.3557e+01],
         ...,
         [-8.3446e-07, -2

In [17]:
indices[0]

tensor([ 9,  6, 15, 10, 13, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 

In [19]:
def to_text(indice, vocab):
    import data_loader as data_loader
    # This method converto index to word to show the summarization result.
    lines = []

    for i in range(len(indice)):
        line = []
        for j in range(len(indice[i])):
            index = indice[i][j]

            if index == data_loader.EOS:
                # line += ['<EOS>']
                break
            else:
                line += [vocab.convert(index)]

        line = ' '.join(line)
        lines += [line]

    return lines


In [23]:
indices.size()

torch.Size([4, 255])

In [24]:
y_hats.size()

torch.Size([4, 255, 11946])

In [43]:
[vocab.tgt_tokenizer.idx2txt[i] for i in indices[3].tolist()]

['선포합니다.',
 '질의하실',
 '들으시고',
 '본회의',
 '시설',
 '바랍니다.',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 'unk_',
 

In [39]:
indices[0].tolist()

[9,
 6,
 15,
 10,
 13,
 14,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [9]:
vocab.tgt_tokenizer.idx2txt[39]

'먼저'